In [22]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from collections import OrderedDict as odict
from graphviz import Digraph
from tabulate import tabulate

In [23]:
# Dependencies for transition probabilities
previous_G = {
    'r1' : ['r2', 'r3'],
    'r2' : ['r1', 'r4'],
    'r3' : ['r1', 'r7'],
    'r4' : ['r2', 'r8'],
    'r5' : ['r6', 'r9', 'c3'],
    'r6' : ['r5', 'c3'],
    'r7' : ['r3', 'c1'],
    'r8' : ['r4', 'r9'],
    'r9' : ['r5', 'r8', 'r13'],
    'r10': ['c3'],
    'r11': ['c3'],
    'r12': ['r22', 'outside'],
    'r13': ['r9', 'r24'],
    'r14': ['r24'],
    'r15': ['c3'],
    'r16': ['c3'],
    'r17': ['c3'],
    'r18': ['c3'],
    'r19': ['c3'],
    'r20': ['c3'],
    'r21': ['c3'],
    'r22': ['r12', 'r25'],
    'r23': ['r24'],
    'r24': ['r13', 'r14', 'r23'],
    'r25': ['r22', 'r26'],
    'r26': ['r25', 'r27'],
    'r27': ['r26', 'r32'],
    'r28': ['c4'],
    'r29': ['r30', 'c4'],
    'r30': ['r29'],
    'r31': ['r32'],
    'r32': ['r27', 'r31', 'r33'],
    'r33': ['r32'],
    'r34': ['c2'],
    'r35': ['c4'],
    'c1' : ['r7', 'r25', 'c2'],
    'c2' : ['r34', 'c1', 'c4'],
    'c3' : ['r5', 'r6', 'r10', 'r11', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'o1'],
    'c4' : ['r28', 'r29', 'r35', 'c2', 'o1'],
    'o1' : ['c3', 'c4'],
    'outside': ['r12']
}

# This just adds itself to all rooms
for i in previous_G.keys():
    previous_G[i] += [i]
    
# Sensor locations
urel_sens_loc = {
    'unreliable_sensor1': 'o1',
    'unreliable_sensor2': 'c3',
    'unreliable_sensor3': 'r1',
    'unreliable_sensor4': 'r24'
}

rel_sens_loc = {
    'reliable_sensor1': 'r16',
    'reliable_sensor2': 'r5',
    'reliable_sensor3': 'r25',
    'reliable_sensor4': 'r31'
}

door_sens_loc = {
    'door_sensor1': ['r8', 'r9'],
    'door_sensor2': ['c1', 'c2'],
    'door_sensor3': ['r26', 'r27'],
    'door_sensor4': ['r35', 'c4']
}

all_sensors={
    'unreliable_sensor1': 'o1',
    'unreliable_sensor2': 'c3',
    'unreliable_sensor3': 'r1',
    'unreliable_sensor4': 'r24',
    'reliable_sensor1': 'r16',
    'reliable_sensor2': 'r5',
    'reliable_sensor3': 'r25',
    'reliable_sensor4': 'r31',
    'door_sensor1': ['r8', 'r9'],
    'door_sensor2': ['c1', 'c2'],
    'door_sensor3': ['r26', 'r27'],
    'door_sensor4': ['r35', 'c4']
}


In [30]:
#Learn the outcomespace

def learn_outcome_space(data):
    outcomeSpace=dict()
    for i in data.keys():
        outcomeSpace[i]=tuple(np.unique(data[i]))
    return outcomeSpace
data = pd.read_csv('data.csv')
data_numpy = data.to_numpy()
data_cols = list(data.columns)
#first column is index in excel
del data[data.columns[0]]

#We are going to consider just if a room is empty or not (we don't care about the exact number of people)
room_columns=[]
for i in range(1,36):
    room_columns.append('r'+str(i))


all_rooms = list(previous_G.keys())
# print(all_rooms)

data_processed = {}
#room_columns=tuple(room_columns)
for i in all_rooms + list(door_sens_loc.keys()):
    data_processed[i] = (data_numpy[:,data_cols.index(i)] > 0)
    # data[i]=data[i].replace(to_replace=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],value=1)    
    # data[i].where(~(data.i!=0),other=1,inplace=True)

# Might need to do something if "None" is returned
for i in list(urel_sens_loc.keys()) + list(rel_sens_loc.keys()):
    data_processed[i] = (data_numpy[:,data_cols.index(i)] == "motion")

outcomeSpace = learn_outcome_space(data_processed)

In [25]:
def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

In [36]:
def allEqualThisIndex(dict_of_arrays, **fixed_vars):
    """
    Helper function to create a boolean index vector into a tabular data structure,
    such that we return True only for rows of the table where, e.g.
    column_a=fixed_vars['column_a'] and column_b=fixed_vars['column_b'].
    
    This is a simple task, but it's not *quite* obvious
    for various obscure technical reasons.
    
    It is perhaps best explained by an example.
    
    >>> all_equal_this_index(
    ...    {'X': [1, 1, 0], Y: [1, 0, 1]},
    ...    X=1,
    ...    Y=1
    ... )
    [True, False, False]
    """
    # base index is a boolean vector, everywhere true
    first_array = dict_of_arrays[list(dict_of_arrays.keys())[0]]
    index = np.ones_like(first_array, dtype=np.bool_)
    for var_name, var_val in fixed_vars.items():
        index = index & (np.asarray(dict_of_arrays[var_name])==var_val)
    return index

def estProbs(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    # Smoothing
    alpha = 1
    prob_table = odict()
    
    # Changed to only output the probability that there are people in the room p, and so P(0) = 1 - p
    # This makes tables much smaller and keeps the exact same information since outcome space is binary
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        #print(parent_vars)
        parent_index = allEqualThisIndex(data, **parent_vars)
        ########we care for the previous state only, so we delete the last row#########
        parent_index=parent_index[:-1]
        #print('parent_index',len(parent_index),parent_index)
        #print('var_outcome:',var_outcome)
        var_index = (np.asarray(data[var_name])==1)[1:]
        ########we need to consider from the second state, so we delete the first row#########
        #print('var_index',len(var_index),var_index)
        prob_table[tuple(list(parent_combination))] = ((var_index & parent_index).sum()+alpha)/(parent_index.sum() + alpha*len(var_outcomes))

    return {'dom': tuple(list(parent_names)), 'table': prob_table}


In [37]:
# Some important assumptions for the emission probabilities:
# -We are considering just the motion detection of a sensor door
# -We are not going to consider time for now (just put an if condition in r12 
#  for the first minutes adn for the last minutes)

# Doors sensor probabilities are calculated as P(ds > 1 | r_i, r_j) for now
# This is something we can probably improve by involving time and stuff, but this should work for now


In [38]:
# Get all transition probabilities
tran_prob_table=odict()
for present, previous in previous_G.items():
    tran_prob_table[present]= estProbs(data_processed, present, previous, outcomeSpace)
    
emis_prob_table=odict()
for sensor, location in all_sensors.items():
    if type(location) == str: location = [location]
    emis_prob_table[sensor]= estProbs(data_processed, sensor, location, outcomeSpace)

for k in emis_prob_table.keys():
    print("Table for", k)
    printFactor(emis_prob_table[k])
    print()

# for k in tran_prob_table.keys():
#     print("Table for", k)
#     printFactor(tran_prob_table[k])
#     print()

Table for unreliable_sensor1
|   o1 |       Pr |
|------+----------|
|    0 | 0.227692 |
|    1 | 0.833333 |

Table for unreliable_sensor2
|   c3 |       Pr |
|------+----------|
|    0 | 0.267688 |
|    1 | 0.752178 |

Table for unreliable_sensor3
|   r1 |       Pr |
|------+----------|
|    0 | 0.190202 |
|    1 | 0.913426 |

Table for unreliable_sensor4
|   r24 |       Pr |
|-------+----------|
|     0 | 0.16601  |
|     1 | 0.512397 |

Table for reliable_sensor1
|   r16 |        Pr |
|-------+-----------|
|     0 | 0.0396367 |
|     1 | 0.970662  |

Table for reliable_sensor2
|   r5 |        Pr |
|------+-----------|
|    0 | 0.0830517 |
|    1 | 0.66967   |

Table for reliable_sensor3
|   r25 |        Pr |
|-------+-----------|
|     0 | 0.0996764 |
|     1 | 0.838184  |

Table for reliable_sensor4
|   r31 |        Pr |
|-------+-----------|
|     0 | 0.0452675 |
|     1 | 0.972246  |

Table for door_sensor1
|   r8 |   r9 |         Pr |
|------+------+------------|
|    0 |    0 |

In [75]:
#Get the robot accuracy, just considering if there is people in a room or not
robot=['robot1','robot2']
prob_robots=[]
for r in robot:    
    #prob of robots
    count=0
    col_robot=data_cols.index(r)
    for i in range(data_numpy.shape[0]):
        #get the room seen by the robot
        s_room=data_numpy[:,col_robot][i].split(',')[0].partition("'")[2].partition("'")[0]
        #get the number of people seen by the robot
        n_people=int(data_numpy[:,col_robot][i].split(',')[1].strip().partition(')')[0])
        if n_people==0:
            n_people=False
        else:
            n_people=True

        #We are just going to consider if the robot sees people or not (we are not going to consider the exact number)
        if data_processed[s_room][i]==n_people:
            count +=1
    prob_robots.append(count/data_numpy.shape[0])
prob_robots

[1.0, 1.0]

In [ ]:
#Given that robots are precise, if we see a robot we are going to decide that there are people in the
#indicated area for sure.
#When a sensor is broken, just choose the highest probability between the possible values that
#the broken sensor could take